In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
working_directory = 'C:/cnolan-thesis/' #set location using back slashes

os.chdir(working_directory)

print("Current working directory: {0}".format(os.getcwd()))


def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
            output_path = os.makedirs(directory)
            print(output_path)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        

# Folder where outputs will be saved (by default a folder within the working directory) 
createFolder('./output/') 
output_path = working_directory +'./output/'


In [ ]:
final_data_all_vars = final_data.dropna(how = 'any',subset=['log_scope1', 'log_scope2', 'log_total_emissions', 'log_energy_consumption', 'change_scope1', 'change_scope2', 'change_total_emissions', 'change_energy_consumption', 'scope1_int', 'scope2_int', 'total_emissions_int', 'energy_consumption_int', 'logsize', 'bm', 'roe', 'leverage', 'investa', 'logppe', 'salesgr', 'epsgr'])
final_data_all_vars = final_data_all_vars.reset_index(drop=True)

final_data_all_determinants = final_data.dropna(how = 'any',subset= ['logsize', 'bm', 'roe', 'leverage', 'investa', 'logppe', 'salesgr', 'epsgr'])
final_data_all_determinants = final_data_all_determinants.dropna(how='all',subset=['log_scope1', 'log_scope2', 'log_total_emissions', 'log_energy_consumption', 'change_scope1', 'change_scope2', 'change_total_emissions', 'change_energy_consumption', 'scope1_int', 'scope2_int', 'total_emissions_int', 'energy_consumption_int'])
final_data_all_determinants = final_data_all_determinants.reset_index(drop=True)

# Observations with no missing values

In [ ]:
print('Number of Observations:')
print (len(final_data_all_vars))

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(final_data_all_vars)


# Observations with no determinant values

I.E some missing carbon variables within the firm year observation

In [ ]:
print('Number of Observations:')
print (len(final_data_all_determinants))

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(final_data_all_determinants)